# Coronavirus Tracker 

### Below notebook presents the current coronavirus situation globally along with detailed analysis for the US

### Disclaimer
##### The following datasets have been recorded from: 
 * https://raw.githubusercontent.com/CSSEGISandData/COVID-19
 * https://www.worldometers.info/coronavirus/

These datasets are updated on a daily basis by the Johns Hopkins Univerity

## Some quick and helpful reference:

* https://www.cdc.gov/coronavirus/2019-ncov/faq.html
* https://www.nhs.uk/conditions/coronavirus-covid-19/
        

In [107]:
from __future__ import print_function

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
from collections import namedtuple

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [108]:
# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [109]:
url1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
df_r = pd.read_csv(url1, index_col=0)

url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_c = pd.read_csv(url2, index_col=0)

url3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_d = pd.read_csv(url3, index_col=0)


In [110]:
df_r.reset_index(inplace=True)

In [111]:
df_c.reset_index(inplace=True)


In [112]:
df_d.reset_index(inplace=True)


In [113]:
df_c.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
df_r.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
df_d.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)


In [114]:
df_confirmed = df_c.melt(id_vars=['province_state','country','Lat','Long'],
                    var_name='Date',
                    value_name='total_confrimed')

In [115]:
df_recover = df_r.melt(id_vars=['province_state','country','Lat','Long'],
                    var_name='Date',
                    value_name='total_recover')

In [116]:
df_deaths = df_d.melt(id_vars=['province_state','country','Lat','Long'],
                    var_name='Date',
                    value_name='total_deaths')

# Earliest confirmed cases of Covid-19 by top 5 countries

In [117]:
df_confirmed.head()

,province_state,country,Lat,Long,Date,total_confrimed
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


# Latest confirmed cases of Covid-19 by top 5 countries

In [118]:
#latest cases
df_confirmed.tail()

,province_state,country,Lat,Long,Date,total_confrimed
43885,NaN,Sao Tome and Principe,0.186360,6.613081,7/4/20,719
43886,NaN,Yemen,15.552727,48.516388,7/4/20,1248
43887,NaN,Comoros,-11.645500,43.333300,7/4/20,309
43888,NaN,Tajikistan,38.861034,71.276093,7/4/20,6159
43889,NaN,Lesotho,-29.609988,28.233608,7/4/20,35


In [119]:
df_confirmed.province_state.fillna('No Info', inplace=True)
df_deaths.province_state.fillna('No Info', inplace=True)
df_recover.province_state.fillna('No Info', inplace=True)

In [120]:
df_confirmed['key'] = df_confirmed['province_state'].astype(str)+'-'+df_confirmed['country'].astype(str)+'-'+df_confirmed['Date'].astype(str)
df_recover['key'] = df_recover['province_state'].astype(str) + '-'+df_recover['country'].astype(str)+'-'+df_recover['Date'].astype(str)
df_deaths['key'] = df_deaths['province_state'].astype(str) + '-'+ df_deaths['country'].astype(str) + '-' + df_deaths['Date'].astype(str)

In [121]:
df_1= df_confirmed[['province_state','country','Lat','Long','Date','total_confrimed','key']].merge(df_deaths[['total_deaths','key']], on='key', how='left')
df_2 = df_1[['province_state','country','Lat','Long','Date','total_confrimed','total_deaths','key']].merge(df_recover[['total_recover','key']],on='key',how='left')


In [122]:
df = df_2.drop(columns=['key'])

In [123]:
df.total_confrimed.fillna(0,inplace=True)
df.total_deaths.fillna(0,inplace=True)
df.total_recover.fillna(0,inplace=True)

In [124]:
#create the region column as well
who_region = pd.read_csv('who_region.csv')
who_region.rename(columns={'Country/Region':'country','WHO Region':'region'},inplace= True)
df = df[['province_state','country','Lat','Long','Date','total_confrimed','total_deaths','total_recover']].merge(who_region[['country','region']],on='country',how='left')

In [125]:
# Reading worlodmeters data for an aggregated view

In [126]:
import json
# datetime oprations
from datetime import timedelta
# to get web contents
from urllib.request import urlopen

# for numerical analyiss
import numpy as np
# to store and process data in dataframe
import pandas as pd
import matplotlib.pyplot as plt
# advanced ploting
import seaborn as sns

# interactive visualization
import plotly.express as px
import plotly.graph_objs as go
# import plotly.figure_factory as ff
from plotly.subplots import make_subplots
# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [127]:
import covid_daily
agg_data = covid_daily.overview(as_json=False)

In [128]:
agg_data = agg_data.drop(columns=['1 Caseevery X ppl','1 Deathevery X ppl','1 Testevery X ppl'])
agg_data.rename(columns={'Country,Other':'country','Serious,Critical':'serious_critical',
                   'Tests/1M pop':'tests_per_1M_popluataion',
                   'TotCases/1M pop':'total_cases_per_1M_popluataion',
                   'Deaths/1M pop':'deaths_per_1M_population'}, inplace=True)


In [129]:
agg_data_sorted = agg_data.sort_values('TotalCases',ascending=False)

In [130]:
#clean data at global level
agg_data_sorted = agg_data_sorted.iloc[2:]

In [131]:
#create the region column as well
who_region = pd.read_csv('who_regionworldometer.csv')
df_final = agg_data_sorted[['country','TotalCases','NewCases','TotalDeaths','NewDeaths','TotalRecovered','NewRecovered','ActiveCases','serious_critical','total_cases_per_1M_popluataion','deaths_per_1M_population','TotalTests','tests_per_1M_popluataion','Population']].merge(who_region[['country','region']],on='country',how='left')


# Interactive Case level view

In [132]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
@interact
def show_case(column=['TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths','ActiveCases'], 
                            x=(10,10000,10)):
    return df_final.loc[df_final[column] > x]

interactive(children=(Dropdown(description='column', options=('TotalCases', 'NewCases', 'TotalDeaths', 'NewDea…

In [133]:
import qgrid
qgrid.show_grid(df_final,show_toolbar=True)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Analysis and some Important plots

In [134]:
def plot_hbar(df, col, n, hover_data=[]):
    fig = px.bar(df.sort_values(col).tail(n),
                 x=col, y="country", color='region',height=700,
                 text=col, orientation='h', width=1050, hover_data=hover_data,
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.update_layout(title=col, xaxis_title="", yaxis_title="", 
                      font=dict(size=15),
                      yaxis_categoryorder = 'total ascending'
                      )
    fig.show()

In [135]:
plot_hbar(df_final, 'TotalCases', 15)

In [136]:
plot_hbar(df_final, 'NewCases', 15)

In [137]:
plot_hbar(df_final, 'TotalDeaths', 15)

In [138]:
plot_hbar(df_final, 'TotalRecovered', 15)


In [139]:
plot_hbar(df_final, 'ActiveCases', 15)



In [140]:
plot_hbar(df_final, 'serious_critical', 15)



In [141]:
# fig = px.scatter(df_final.head(15),x='country',y='TotalCases',size='TotalCases',height=700,width=1000,
#                  color='country',hover_name='country',size_max=60,
#                  title='Total Cases by Country')
# fig.update_layout()
# fig.show()

In [142]:
# fig = px.scatter(df_final.head(15),x='country',y='ActiveCases',size='ActiveCases',
#                 color='country',hover_name='country',size_max=60,title='Total Active Cases by Country')
# fig.update_layout()
# fig.show()

# Total Deaths vs Confirmed cases(log scale)

In [143]:
fig = px.scatter(df_final.sort_values('TotalDeaths', ascending=False).iloc[:20, :], 
                 x='TotalCases', y='TotalDeaths', color='country', size='TotalCases', 
                 height=600,width=1200, text='country', log_x=True, log_y=True
                 )
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False,font=dict(size=15))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [144]:
def plot_cases_coutry(country):
    labels = ['confirmed', 'deaths' ,'receovered']
    colors = ['blue','red','green']
    mode_size = [6,8,10]
    line_size = [4,5,6]
    
    df_lst = [df_c , df_d ,df_r]
    
    fig = go.Figure()
    
    for i , df in enumerate(df_lst):
        if country == 'US' or country == 'us':
            x_data = np.array(list(df.iloc[:,4:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis=0)
        else:
            x_data = np.array(list(df.iloc[:,4:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,4:]),axis=0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, 
                                 mode ='lines+markers',
                                 name=labels[i],line=dict(color=colors[i],width=line_size[i]), 
                                 connectgaps=True, 
                                 text = "Total" + str(labels[i]) + ": " + str(y_data[-1])
            ))
                                                    
    fig.show()

# Case Distribution Rate at Country level

In [145]:
# plot_cases_coutry('China')
interact(plot_cases_coutry,country='US');#first default plot for US

interactive(children=(Text(value='US', description='country'), Output()), _dom_classes=('widget-interact',))

# Coronavirus heat map

In [146]:
import folium
world_map = folium.Map(location=[11,0],tiles= 'cartodbpositron',zoom_start=2,max_zoom=6,
                      min_zoom=2)
for i in range(len(df_c)):
    folium.Circle(
    location=[df_c.iloc[i]['Lat'],df_c.iloc[i]['Long']],
    fill=True,
    radius = (int((np.log(df_c.iloc[i,-1]+1.000001)))+0.2)*50000,
    fill_color='yellow',
    color='green',
    tooltip="<div style='margin:0; background-color:blue; color: white;'>"+
                 "<h4 style='text-align:center; font-weight:bold;'>"+df_c.iloc[i]['country'] + "</h4>"
                 "<hr style='margin:10px;color:white;'>"+
                 "<ul style='color:white;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                     "<li>Confirmed cases: "+str(df_c.iloc[i,-1])+"</li>"+
                     "<li>Total Deaths: "+str(df_d.iloc[i,-1])+"</li>"+
                     "<li>Death Rate: "+str(np.round(df_d.iloc[i,-1]/(df_c.iloc[i,-1]+1.00001)*100,2))+"</li>"+
                  "</ul></div>"
    ).add_to(world_map)
    
world_map

# Countries with no active cases (Updated daily)

In [147]:
from tabulate import tabulate
no_active = df_final[df_final['ActiveCases']==0]
no_active = no_active.sort_values('TotalCases', ascending=False)
no_active.reset_index(drop=True)

,country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,serious_critical,total_cases_per_1M_popluataion,deaths_per_1M_population,TotalTests,tests_per_1M_popluataion,Population,region
0,Spain,297625,0,28385,0,0,0,0,617,6366,607,5448984,116543,46755070,Europe
1,UK,285416,516,44220,22,0,0,0,231,4204,651,10505758,154745,67890913,Europe
2,Sweden,71419,0,5420,0,0,0,0,124,7071,537,519113,51397,10100100,Europe
3,Netherlands,50621,73,6127,1,0,0,0,18,2954,358,616376,35971,17135450,Europe
4,San Marino,698,0,42,0,656,0,0,0,20571,1238,5729,168838,33932,Europe
5,Isle of Man,336,0,24,0,312,0,0,0,3951,282,6600,77611,85039,NaN
6,Faeroe Islands,187,0,0,0,187,0,0,0,3827,0,15899,325359,48866,NaN
7,Bermuda,146,0,9,0,137,0,0,1,2344,145,12046,193435,62274,NaN
8,Brunei,141,0,3,0,138,0,0,0,322,7,29841,68204,437527,Western Pacific
9,Sint Maarten,78,0,15,0,63,0,0,0,1819,350,500,11660,42881,NaN
